<a href="https://colab.research.google.com/github/veethikasingh/we-r-no-angels/blob/main/Crime_data_spark_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [929 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2,627 kB]
Get:13 http://archive.ubuntu.com/ub

In [3]:
from google.colab import files
uploaded = files.upload()

Saving Crime_Data_from_2020_to_Present_20240611 clean.csv to Crime_Data_from_2020_to_Present_20240611 clean.csv


In [4]:
# Import dependencies
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

### Extract the crime Data

In [5]:
# Read the data into a DataFrame
from pyspark import SparkFiles
crimes_df = spark.read.csv(SparkFiles.get("/content/Crime_Data_from_2020_to_Present_20240611 clean.csv"), sep=",", header=True)
crimes_df.show()





+---------+-------------+-------------+--------+----+-----------+-----------+--------+------+--------------------+--------------------+--------+--------+------------+---------+--------------------+--------------+--------------------+------+------------+--------+--------+--------+--------+--------------------+--------------------+-------+---------+
|    DR_NO|    Date Rptd|     DATE OCC|TIME OCC|AREA|  AREA NAME|Rpt Dist No|Part 1-2|Crm Cd|         Crm Cd Desc|             Mocodes|Vict Age|Vict Sex|Vict Descent|Premis Cd|         Premis Desc|Weapon Used Cd|         Weapon Desc|Status| Status Desc|Crm Cd 1|Crm Cd 2|Crm Cd 3|Crm Cd 4|            LOCATION|        Cross Street|    LAT|      LON|
+---------+-------------+-------------+--------+----+-----------+-----------+--------+------+--------------------+--------------------+--------+--------+------------+---------+--------------------+--------------+--------------------+------+------------+--------+--------+--------+--------+-----------

In [6]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [7]:
# Rename Columns
df = crimes_df.withColumnRenamed("AREA NAME",
                                  "District").withColumnRenamed("Crm Cd Desc",
                                                          "Crimes")



In [8]:
df.show()

+---------+-------------+-------------+--------+----+-----------+-----------+--------+------+--------------------+--------------------+--------+--------+------------+---------+--------------------+--------------+--------------------+------+------------+--------+--------+--------+--------+--------------------+--------------------+-------+---------+
|    DR_NO|    Date Rptd|     DATE OCC|TIME OCC|AREA|   District|Rpt Dist No|Part 1-2|Crm Cd|              Crimes|             Mocodes|Vict Age|Vict Sex|Vict Descent|Premis Cd|         Premis Desc|Weapon Used Cd|         Weapon Desc|Status| Status Desc|Crm Cd 1|Crm Cd 2|Crm Cd 3|Crm Cd 4|            LOCATION|        Cross Street|    LAT|      LON|
+---------+-------------+-------------+--------+----+-----------+-----------+--------+------+--------------------+--------------------+--------+--------+------------+---------+--------------------+--------------+--------------------+------+------------+--------+--------+--------+--------+-----------

In [9]:
#Create temp view
df.createOrReplaceTempView('crimesdata')

In [10]:
# Get highset crime Area/District
df_sum = spark.sql("SELECT District, count(*) as crimecount FROM crimesdata GROUP BY District Order by crimecount desc")
df_sum.show()

+-----------+----------+
|   District|crimecount|
+-----------+----------+
|    Central|     63827|
|77th Street|     59240|
|    Pacific|     55046|
|  Southwest|     53590|
|  Hollywood|     49344|
|N Hollywood|     48081|
|  Southeast|     47710|
|    Olympic|     47409|
|     Newton|     46586|
|   Wilshire|     45208|
|    Rampart|     43966|
|    West LA|     43127|
|   Van Nuys|     40427|
|  Northeast|     40316|
|West Valley|     40099|
|     Harbor|     39160|
| Devonshire|     39159|
|    Topanga|     39061|
|    Mission|     37841|
| Hollenbeck|     34800|
+-----------+----------+
only showing top 20 rows



In [11]:
df.printSchema()

root
 |-- DR_NO: string (nullable = true)
 |-- Date Rptd: string (nullable = true)
 |-- DATE OCC: string (nullable = true)
 |-- TIME OCC: string (nullable = true)
 |-- AREA: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Rpt Dist No: string (nullable = true)
 |-- Part 1-2: string (nullable = true)
 |-- Crm Cd: string (nullable = true)
 |-- Crimes: string (nullable = true)
 |-- Mocodes: string (nullable = true)
 |-- Vict Age: string (nullable = true)
 |-- Vict Sex: string (nullable = true)
 |-- Vict Descent: string (nullable = true)
 |-- Premis Cd: string (nullable = true)
 |-- Premis Desc: string (nullable = true)
 |-- Weapon Used Cd: string (nullable = true)
 |-- Weapon Desc: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Status Desc: string (nullable = true)
 |-- Crm Cd 1: string (nullable = true)
 |-- Crm Cd 2: string (nullable = true)
 |-- Crm Cd 3: string (nullable = true)
 |-- Crm Cd 4: string (nullable = true)
 |-- LOCATION: string (nu

In [12]:
# Get highest Crime type
start_time = time.time()
df_sum_area = spark.sql("SELECT  Crimes, count(*) as crimecount FROM crimesdata GROUP BY  Crimes Order by  crimecount desc")
df_sum_area.show()

print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+----------+
|              Crimes|crimecount|
+--------------------+----------+
|    VEHICLE - STOLEN|    102820|
|BATTERY - SIMPLE ...|     74185|
|BURGLARY FROM VEH...|     58522|
|   THEFT OF IDENTITY|     58493|
|            BURGLARY|     57512|
|VANDALISM - FELON...|     57379|
|ASSAULT WITH DEAD...|     53187|
|THEFT PLAIN - PET...|     48348|
|INTIMATE PARTNER ...|     46427|
|THEFT FROM MOTOR ...|     36835|
|THEFT FROM MOTOR ...|     34009|
|             ROBBERY|     31975|
|THEFT-GRAND ($950...|     31800|
|SHOPLIFTING - PET...|     25714|
|VANDALISM - MISDE...|     24247|
|CRIMINAL THREATS ...|     19179|
|         TRESPASSING|     14546|
|     BRANDISH WEAPON|     14445|
|INTIMATE PARTNER ...|     12627|
|VIOLATION OF REST...|     11586|
+--------------------+----------+
only showing top 20 rows

--- 6.843285083770752 seconds ---


<IPython.core.display.Javascript object>